In [1]:
from langchain_openai import ChatOpenAI

from extract import (
    extract_function_names,
    extract_function_by_name,
    extract_imports,
)

from llm_calls import (
    llm_generate_unit_tests,
    llm_generate_function_from_unit_tests,
    llm_find_bug,
    llm_fix_bug,
)

from run_tests import (
    find_first_failing_unit_test,
    run_unit_test,
)

from function_code_with_unit_tests import FunctionCodeWithUnitTests

llm = ChatOpenAI(temperature=0, model="gpt-4o")

# query = "Extract emails (including instances of 'name at email dot com') from a text."
# query = "Swap the words 'cat' and 'dog' in the text."

query = "Please replace all instances of 'Alice' with 'Bob', 'Bob' with 'Carol', and 'Carol' with 'Alice'. \
    The output string should always capitalise these names, even if they weren't capitalised in the input text."

In [2]:
unit_test_code_as_string = llm_generate_unit_tests(llm, query)
print(unit_test_code_as_string)

# Function to test: replace_names(text: str) -> str

import re

def test_replace_names_basic():
    input_text = "Alice went to see Bob and Carol."
    expected_output = "Bob went to see Carol and Alice."
    assert replace_names(input_text) == expected_output, f"Expected: {expected_output}, but got: {replace_names(input_text)}"

def test_replace_names_case_insensitive():
    input_text = "alice went to see bob and carol."
    expected_output = "Bob went to see Carol and Alice."
    assert replace_names(input_text) == expected_output, f"Expected: {expected_output}, but got: {replace_names(input_text)}"

def test_replace_names_mixed_case():
    input_text = "Alice went to see bob and CAROL."
    expected_output = "Bob went to see Carol and Alice."
    assert replace_names(input_text) == expected_output, f"Expected: {expected_output}, but got: {replace_names(input_text)}"

def test_replace_names_multiple_instances():
    input_text = "Alice and Bob went to see Carol. Bob and Carol were h

In [3]:
unit_test_function_names = extract_function_names(code_string=unit_test_code_as_string)
unit_test_function_names

['test_replace_names_basic',
 'test_replace_names_case_insensitive',
 'test_replace_names_mixed_case',
 'test_replace_names_multiple_instances',
 'test_replace_names_no_names',
 'test_replace_names_empty_string',
 'test_replace_names_special_characters',
 'test_replace_names_unicode_characters',
 'test_replace_names_with_numbers',
 'test_replace_names_with_punctuation',
 'test_replace_names_with_newlines',
 'test_replace_names_with_tabs',
 'test_replace_names_with_mixed_content',
 'test_replace_names_with_anchors',
 'test_replace_names_with_lookahead_lookbehind']

In [4]:
unit_test_imports = extract_imports(unit_test_code_as_string)
unit_test_imports

['import re']

In [5]:
target_function_code_as_string = llm_generate_function_from_unit_tests(
    llm=llm,
    unit_test_code_as_string=unit_test_code_as_string,
    query=query,
)
print(target_function_code_as_string)

import re

def replace_names(text: str) -> str:
    """
    Replace all instances of 'Alice' with 'Bob', 'Bob' with 'Carol', and 'Carol' with 'Alice'.
    The output string should always capitalize these names, even if they weren't capitalized in the input text.

    :param text: The input text containing names to be replaced.
    :return: The text with names replaced and capitalized.
    """
    def replace(match):
        name = match.group(0).lower()
        if name == 'alice':
            return 'Bob'
        elif name == 'bob':
            return 'Carol'
        elif name == 'carol':
            return 'Alice'
        return match.group(0)

    return re.sub(r'\b(Alice|Bob|Carol)\b', replace, text, flags=re.IGNORECASE)



In [6]:
function_code_with_unit_tests = FunctionCodeWithUnitTests(
    function_code=[target_function_code_as_string],
    unit_tests={
        name: [
            extract_function_by_name(
                code_string=unit_test_code_as_string, function_name=name
            )
        ]
        for name in unit_test_function_names
    },
    unit_test_imports="\n".join(unit_test_imports)
)


In [7]:
for i in range(10):

    first_failing_unit_test_message = find_first_failing_unit_test(
        function_code_with_unit_tests=function_code_with_unit_tests
    )

    first_failing_unit_test_message.print_message_details()

    if first_failing_unit_test_message.test_pass_status:
        print("All tests are passing")
        break

    print("Ask llm if the function or unit test needs to be updated.")
    bug_report_dict = llm_find_bug(
        llm=llm,
        query=query,
        function_code_with_unit_tests=function_code_with_unit_tests,
        first_failing_unit_test_message=first_failing_unit_test_message,
    )

    if bug_report_dict["Likely Bug Location"] == "Unit Test":
        
        print("The llm said the unit test should be updated.\nAsk the llm to fix the unit test.\n")
        updated_unit_test_code = llm_fix_bug(
            llm=llm,
            query=query,
            bug_report_dict=bug_report_dict,
            function_code_with_unit_tests=function_code_with_unit_tests,
            first_failing_unit_test_message=first_failing_unit_test_message,
        )
        
        print("Unit test before update;")
        print(function_code_with_unit_tests.get_unit_test_code(first_failing_unit_test_message.unit_test_name) + "\n")
        
        function_code_with_unit_tests.replace_unit_test(
            unit_test_name=first_failing_unit_test_message.unit_test_name,
            new_unit_test_code=updated_unit_test_code,
        )
        
        print("Unit test after update;")
        print(function_code_with_unit_tests.get_unit_test_code(first_failing_unit_test_message.unit_test_name) + "\n")

    elif bug_report_dict["Likely Bug Location"] == "Function":
        
        print("The llm said the function should be updated.\nAsk the llm to fix the function.\n")
        updated_function_code = llm_fix_bug(
            llm=llm,
            query=query,
            bug_report_dict=bug_report_dict,
            function_code_with_unit_tests=function_code_with_unit_tests,
            first_failing_unit_test_message=first_failing_unit_test_message,
        )
        
        print("Function before update;")
        print(function_code_with_unit_tests.get_function_code() + "\n")
        
        function_code_with_unit_tests.replace_function_code(
            new_function_code=updated_function_code,
        )
        
        print("Function after update;")
        print(function_code_with_unit_tests.get_function_code() + "\n")
        
    else:
        raise ValueError("bug_report_dict keys not as expected")

    unit_test_message = run_unit_test(
        function_code_with_unit_tests=function_code_with_unit_tests,
        unit_test_name=first_failing_unit_test_message.unit_test_name,
    )
    
    if unit_test_message.test_pass_status:
    
        print(f"Unit test {first_failing_unit_test_message.unit_test_name}() now passing.")
    
    else:
    
        unit_test_code_to_delete = function_code_with_unit_tests.get_unit_test_code(
            first_failing_unit_test_message.unit_test_name
        )
        print(
            f"Unable to make this unit test pass\n{unit_test_code_to_delete}It will be deleted\n"
        )
        function_code_with_unit_tests.delete_unit_test_code(
            first_failing_unit_test_message.unit_test_name
        )
        if bug_report_dict["Likely Bug Location"] == "Function":
            print("Roll back update to function.")
            function_code_with_unit_tests.delete_function_changes()


if first_failing_unit_test_message.test_pass_status == False:
    print("Some tests are still not passing")

Python REPL can execute arbitrary code. Use with caution.


unit test test_replace_names_basic() is currently passing. 0/15 tests pass.
unit test test_replace_names_case_insensitive() is currently passing. 1/15 tests pass.
unit test test_replace_names_mixed_case() is currently passing. 2/15 tests pass.
unit test test_replace_names_multiple_instances() is currently passing. 3/15 tests pass.
unit test test_replace_names_no_names() is currently passing. 4/15 tests pass.
unit test test_replace_names_empty_string() is currently passing. 5/15 tests pass.
unit test test_replace_names_special_characters() is currently passing. 6/15 tests pass.

test_pass_status: False
unit_test_name: test_replace_names_unicode_characters
output: Assertion failed: Expected: Bob went to see Carol and Alice., but got: Álice went to see Bób and Càrol.

Ask llm if the function or unit test needs to be updated.
The llm said the unit test should be updated.
Ask the llm to fix the unit test.

Unit test before update;
def test_replace_names_unicode_characters():
    input_text 

In [8]:
import re
def test_extract_cat_and_dog_basic():
    text = 'I have a cat and a dog.'
    pattern = '\\b(cat|dog)\\b'
    matches = re.findall(pattern, text)
    assert matches == ['cat', 'dog'
        ], f"Expected ['cat', 'dog'], but got {matches}"



test_extract_cat_and_dog_basic()

In [9]:
print(function_code_with_unit_tests.get_function_code())

import re

def replace_names(text: str) -> str:
    """
    Replace all instances of 'Alice' with 'Bob', 'Bob' with 'Carol', and 'Carol' with 'Alice'.
    The output string should always capitalize these names, even if they weren't capitalized in the input text.

    :param text: The input text containing names to be replaced.
    :return: The text with names replaced and capitalized.
    """
    def replace(match):
        name = match.group(0).lower()
        if name == 'alice':
            return 'Bob'
        elif name == 'bob':
            return 'Carol'
        elif name == 'carol':
            return 'Alice'
        return match.group(0)

    return re.sub(r'(Alice|Bob|Carol)', replace, text, flags=re.IGNORECASE)



In [10]:
for unit_test_name in function_code_with_unit_tests.unit_tests:
    print(
        function_code_with_unit_tests.get_unit_test_code(unit_test_name=unit_test_name)
    )

def test_replace_names_basic():
    input_text = 'Alice went to see Bob and Carol.'
    expected_output = 'Bob went to see Carol and Alice.'
    assert replace_names(input_text
        ) == expected_output, f'Expected: {expected_output}, but got: {replace_names(input_text)}'

def test_replace_names_case_insensitive():
    input_text = 'alice went to see bob and carol.'
    expected_output = 'Bob went to see Carol and Alice.'
    assert replace_names(input_text
        ) == expected_output, f'Expected: {expected_output}, but got: {replace_names(input_text)}'

def test_replace_names_mixed_case():
    input_text = 'Alice went to see bob and CAROL.'
    expected_output = 'Bob went to see Carol and Alice.'
    assert replace_names(input_text
        ) == expected_output, f'Expected: {expected_output}, but got: {replace_names(input_text)}'

def test_replace_names_multiple_instances():
    input_text = (
        'Alice and Bob went to see Carol. Bob and Carol were happy to see Alice.'
       